<div align="center">
  <img src="assets/Day9.png" alt="Databricks 14 Days AI Challenge - Day 09" width="800"/>
</div>

## DAY 9 (17/01/26) – SQL Analytics & Dashboards

### 📚 Learning Objectives
We have successfully built a governed Lakehouse (Bronze -> Silver -> Gold). Now, we wear the "Analyst" hat. Today we focus on extracting insights using **Databricks SQL**:
* **Complex Analytical Queries:** Using Common Table Expressions (CTEs) and Window Functions for trend analysis.
* **Funnel Analysis:** Tracking user conversion rates.
* **Visualization:** Creating professional charts (Line, Bar, Funnel) directly from SQL results.

### 🚀 Strategy: "From Data to Decisions"
We will write three targeted SQL queries to answer C-Suite business questions:
1.  **Revenue Trends:** Are sales growing? (Using Moving Averages).
2.  **Conversion Funnel:** Where are we losing customers? (View-to-Purchase ratios).
3.  **Customer Segmentation:** Who are our VIPs? (RFM Analysis).

###Setup & Context
**Task**: Switch to SQL mode and ensure we are using the correct Catalog/Schema. **Concept**: In a notebook, %sql allows us to run standard ANSI SQL commands.

In [0]:
# Switch to our governed catalog from Day 8
# If you are on Community Edition, use 'hive_metastore'
catalog_name = "course_catalog" 
schema_name = "ecommerce_governed"

print(f"🔄 Setting SQL Context to: {catalog_name}.{schema_name}")

try:
    spark.sql(f"USE CATALOG {catalog_name}")
    spark.sql(f"USE SCHEMA {schema_name}")
except Exception as e:
    # Fallback for Community Edition
    print("⚠️ Using hive_metastore (Community Edition mode)")
    spark.sql("USE CATALOG hive_metastore")
    spark.sql("CREATE SCHEMA IF NOT EXISTS ecommerce_governed")
    spark.sql("USE SCHEMA ecommerce_governed")

###Revenue Trends (Moving Average)
**Task**: Calculate daily revenue and a 7-day moving average. **Concept**: Sales data is noisy. A Moving Average smooths out daily spikes to show the true underlying trend.

In [0]:
%sql
-- 📊 QUERY 1: REVENUE TRENDS & SMOOTHING
-- We use a CTE (Common Table Expression) to aggregate daily totals first,
-- then apply a Window Function to smooth the trend.

WITH daily_sales AS (
    SELECT 
        event_date, 
        SUM(price) as daily_revenue
    FROM silver_events
    WHERE event_type = 'purchase'
    GROUP BY event_date
)
SELECT 
    event_date,
    daily_revenue,
    -- Calculate 7-Day Moving Average
    AVG(daily_revenue) OVER (
        ORDER BY event_date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as revenue_7day_ma
FROM daily_sales
ORDER BY event_date ASC;

-- 💡 VISUALIZATION TIP:
-- 1. Hover over the Table results below.
-- 2. Click the '+' icon -> 'Visualization'.
-- 3. Select 'Line Chart'.
-- 4. X-Axis: event_date, Y-Axis: daily_revenue AND revenue_7day_ma.

Databricks visualization. Run in Databricks to view.

###Product Conversion Funnel
**Task**: Calculate conversion rates by category. **Concept**: A "Funnel" measures efficiency. High views but low purchases indicate a pricing or UX problem.

In [0]:
%sql
-- 📊 QUERY 2: CATEGORY CONVERSION FUNNEL
-- We analyze our Gold 'product_perf' table to see which categories convert best.

SELECT 
    category_code,
    SUM(unique_views) as total_views,
    SUM(unique_purchases) as total_purchases,
    -- Calculate Conversion Rate %
    ROUND((SUM(unique_purchases) / SUM(unique_views)) * 100, 2) as conversion_rate_pct
FROM gold_product_perf
WHERE category_code IS NOT NULL
GROUP BY category_code
HAVING total_views > 100 -- Filter out noise (low traffic categories)
ORDER BY total_purchases DESC
LIMIT 15;

-- 💡 VISUALIZATION TIP:
-- 1. Create a 'Combo Chart'.
-- 2. Bar Series: total_views.
-- 3. Line Series (Right Y-Axis): conversion_rate_pct.
-- This highlights categories with high traffic but poor conversion.

Databricks visualization. Run in Databricks to view.

###Customer Segmentation (RFM Lite)
**Task**: Segment customers into tiers (VIP, Loyal, Regular). **Concept**: Not all customers are equal. We use a CASE statement to bucket users based on their lifetime purchase count.

In [0]:
%sql
-- 📊 QUERY 3: CUSTOMER VALUE SEGMENTATION
-- We define tiers based on purchase frequency.

WITH customer_stats AS (
    SELECT 
        user_id, 
        COUNT(*) as purchase_count, 
        SUM(price) as total_spent
    FROM silver_events 
    WHERE event_type = 'purchase' 
    GROUP BY user_id
)
SELECT
    CASE 
        WHEN purchase_count >= 10 THEN '1. VIP (10+ Orders)'
        WHEN purchase_count >= 5 THEN '2. Loyal (5-9 Orders)'
        ELSE '3. Regular (1-4 Orders)' 
    END as customer_tier,
    
    COUNT(user_id) as total_customers,
    ROUND(AVG(total_spent), 2) as avg_lifetime_value
FROM customer_stats
GROUP BY 1
ORDER BY 1;

-- 💡 VISUALIZATION TIP:
-- 1. Create a 'Pie Chart' or 'Bar Chart'.
-- 2. Keys: customer_tier, Values: total_customers.

Databricks visualization. Run in Databricks to view.

### 🧠 Key Learnings & Takeaways
* **SQL for Analytics:** While PySpark is great for engineering, SQL is often more expressive for "Business Logic" like moving averages and cohort analysis.
* **Window Functions:** We used `AVG(...) OVER (ROWS BETWEEN ...)` to perform sophisticated time-series smoothing without writing complex Python loops.
* **Visualization:** We learned that raw tables are hard to read. By adding **Native Dashboards** (Line Charts, Funnels), we transformed raw rows into actionable business stories.

###VISUALIZATION

<div align="center">
  <img src="assets/day9_dashboard.png" alt="Databricks 14 Days AI Challenge - Day 09"/>
</div>